In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/spotify_millsongdata.csv')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

,0
artist,0
song,0
link,0
text,0


In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,Yello,The Rhythm Divine,You know that my tears \r\nHave kept me awake...
1,Kelly Family,Father's Nose,Why tu llegada rey me hico feliz \r\nWhy tu v...
2,Demi Lovato,Fix A Heart,"It's probably what's best for you, \r\nI only..."
3,Kiss,I Still Love You,You tell me that you're leaving \r\nAnd I'm t...
4,John Mellencamp,Between A Laugh And A Tear,When paradise is no longer fit for you to live...
5,Billie Holiday,I'll Never Smile Again,I'll never smile again \r\nUntil I smile at y...
6,Noa,Ave Maria,Ave Maria \r\nWhere have you been hiding? \r...
7,Indiana Bible College,Void,Verse 1: \r\n \r\nIs This the end is this al...
8,Nirvana,Dazed And Confused,Been dazed and confused for so long \r\nIt's ...
9,John Prine,Daddy's Little Pumpkin,You must be daddy's little pumpkin \r\nI can ...


In [10]:
df['text'][0]

"You know that my tears  \r\nHave kept me awake  \r\nThe longer you're gone  \r\nI'll hunger and shake  \r\nFrom Warsaw to Rome  \r\nI'll wait out of time  \r\nWith you in my heart  \r\nThe rhythm divine  \r\n  \r\nSo won't you come close  \r\nBring this to an end  \r\nWith each winter rose  \r\nMy love I will send  \r\nSo tender the night  \r\nWhen you hold me tight  \r\nWith you in my heart  \r\nThe rhythm divine  \r\n  \r\nWith you in my heart  \r\nThe rhythm divine  \r\n  \r\nYou know that these tears  \r\nHave kept me awake  \r\nThe longer you're gone  \r\nI'll hunger and shake  \r\nFrom Warsaw to Rome  \r\nI'll wait out of time  \r\nWith you in my heart  \r\nThe rhythm divine  \r\nThe rhythm divine  \r\nThe rhythm divine  \r\n  \r\nWith you in my heart  \r\nThe rhythm divine\r\n\r\n"

In [ ]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [15]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [16]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [19]:
similarity[0]

array([1.        , 0.        , 0.03510781, ..., 0.01864003, 0.01771184,
       0.11082249])

In [23]:
df[df['song'] == 'The Rhythm Divine']

,artist,song,text
0,Yello,The Rhythm Divine,you know that my tear have kept me awak the lo...


In [24]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [25]:
recommendation('The Rhythm Divine')

["Ain't Got Rhythm",
 'O Holy Night',
 'Razor',
 'O Holy Night',
 'O Holy Night',
 'Come To My World',
 'Until Dawn',
 'You And Me',
 'Kick Your Butt',
 'O Holy Night',
 'Rhythm Of Romance',
 'Here Is My Heart',
 'Divine Intervention',
 "You're My World",
 'Walking On Air',
 'Oh Yeah',
 'Careless Whisper',
 'Have I Told You Lately',
 'Blacksmith Blues',
 'Close Your Eyes']

In [26]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))